In [ ]:
%matplotlib inline
import sys
sys.path.append("..") # Adds the module to path

# DeepTrack

This notebook gives an overview of the capabilities of DeepTrack and how to model optical systems using predifined classes.

### What is DeepTrack?

DeepTrack is fundamentally permits to create, train and execute particle tracking models.

## 1. Generate images

A central capability is to generate images, which can be used to train and validate the operation of particle tracking models. For example, these images may be like the output of optical systems employed in experiments. 

The main idea of DeepTrack is that any image can be viewed as a series of features. These features take an input image and update it according to some update rule. For example, a feature can add a particle, introduce some noise, or image something through an optical device.

### Features and properties

Features in DeepTrack are classes implementing the class `Feature`. The way a feature updates an image is governed by the values passed to the class constructor. These inputs are converted to [properties](../examples/properties_example.ipynb). For example, a property could be the position of a particle.

In [ ]:
from deeptrack.scatterers import PointParticle

particle = PointParticle(
    position=(0, 0),
    intensity=100,
    position_unit="pixel"
)

The above code creates a feature (a point particle). This feature will always add a point particle at x=0, y=0. For machine learning, it may be more useful to add a particle at a random position. Thsi can be done by passing a lambda function that returns a random pair of numbers.

In [ ]:
import numpy as np

particle = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=100
)

### Resolving an image

To create an image from a feature, just call its method `.resolve()` with an empty ndarray (all elements zero). To update the properties (here, generating a new random position), call the method `.update()`.

In [ ]:
import matplotlib.pyplot as plt

input_image = np.zeros((64, 64))

# Generate a new random position
particle.update()

# Add the particle to the image
output_image = particle.resolve(input_image)

plt.imshow(output_image, cmap='gray')
plt.show()

Notice that the output doesn't look much like a point scatterer. This is because it's not imaged through an optical device. 

Optical devices are also features. They convolve the input image with a pupil function. They also pass optical properties to the features used to generate their input.

In [ ]:
from deeptrack.optics import OpticalDevice

optics = OpticalDevice(
    NA=0.8,
    wavelength=680e-9,
    pixel_size=100e-9
)

# To image a feature, call optics with the feature
imaged_particle = optics(particle)

imaged_particle.update()

output_image = imaged_particle.resolve(input_image)

plt.imshow(output_image, cmap='gray')
plt.show()

#TODO: Not sure how .update() works. Even if I remove it from this cell, it still updates the particle

### Adding more features

Features can be combined (see [features_example](../examples/features_example.ipynb)) using overloaded operators (+, \*, \*\* or ()). Here, we exemplify the add operator (+) and the power opertor (\*\*).

In [ ]:
# The + operator
particle_1 = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=100
)

particle_2 = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=100
)

# two_particles is a new feature that first resolves particle_1 and then particle_2, then images it
two_particles = optics(particle_1 + particle_2)

output_image = two_particles.resolve(input_image)

plt.imshow(output_image, cmap='gray')
plt.show()

In [ ]:
# The ** operator

particle = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=100
)

# five_particles is a feature that resolves five deep copies of particle, then images it
five_particles = optics(particle**5)

output_image = five_particles.resolve(input_image)

plt.imshow(output_image, cmap='gray')
plt.show()

### Zero-padding the input

You may have noticed that particles close to edges wrap around to the other side. This is due to the wrap-around effect of the Fourier transform used when convolving an image with a pupil. To solve this problem, we zero-pad the input. The desired size can be retrieved in the optics.

In [ ]:
optics = OpticalDevice(
    NA=0.8,
    wavelength=680e-9,
    pixel_size=100e-9,
    ROI = (0, 0, 64, 64) # row, column, height, width
)

five_particles = optics(particle**5)

# Input a slightly larger image as zero padding
input_image = np.zeros((96, 96))

output_image = five_particles.resolve(input_image)

plt.imshow(output_image, cmap='gray')
plt.show()

### Adding noise

To make the image more realistic, we can add some noise. Noise can be added before (most commonly) or after applying the optical device.

In [ ]:
from deeptrack.noises import Offset, Poisson

# Add a constant value to the background
offset = Offset(offset=10)

# Introduce Poisson noise to the image
poisson_noise = Poisson(snr=100)

# resolve five particles, add a offset, image it, and finally introduce Poisson noise
noisy_particles = optics(particle**5 + offset) + poisson_noise

output_image = noisy_particles.resolve(input_image)

plt.imshow(output_image, cmap='gray')
plt.show()

## 2. Retrieve information about the image

To train a supervised machine-learning model, labled images are needed. When a features is resolved, it automatically stores the properties of all features used to create the image. This allows us to extract information about the image in order to use them to train machine-learning models.

Here, we extract the position of all the particles and plot them as red crossed on the generated image.

In [ ]:
def get_positions(image):
    # All properties are stored in the `properties` field of the output.
    positions = [property_dict["position"] for property_dict in image.properties if "position" in property_dict]
    return np.array(positions)


noisy_particles.update()
output_image = noisy_particles.resolve(input_image)

positions = get_positions(output_image)

plt.imshow(output_image, cmap='gray')
plt.scatter(positions[:, 0], positions[:, 1], c="r", marker="x")
plt.show();

## 3. Wrap features in generators

Generators are ways to continuously resolve new images, and are the prefered interface to machine learning models. The default generator is defined in the module `generators`. We can also optionally pass a label function that will be called on every image.

In [ ]:
from deeptrack.generators import Generator

generator = Generator().generate(noisy_particles, get_positions, shape=(96, 96))

for _ in range(4):
    # Outputs shape (1, height, width, 1)
    next_image, positions = next(generator)
    plt.imshow(np.squeeze(next_image), cmap='gray')
    plt.scatter(positions[0, :, 0], positions[0, :, 1], c="r", marker="x")
    plt.show()

## 4. Finally, a complete example that also trains a neural-network model

In [ ]:
from deeptrack.scatterers import PointParticle
from deeptrack.noises import Offset, Poisson
from deeptrack.optics import OpticalDevice
from deeptrack.generators import Generator
from deeptrack.models import convolutional

# DEFINE FEATURES
optics = OpticalDevice(NA=0.8, wavelength=680e-9, pixel_size=100e-9, ROI=(0, 0, 64, 64))

particle = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=100
)

offset = Offset(
    offset = lambda: np.random.rand() * 20
)

poisson_noise = Poisson(snr=np.linspace(50, 100))

training_set = optics(particle + offset) + poisson_noise


# DEFINE LABEL FUNCTION
def get_position(image):
    for propertydict in image.properties:
        if "position" in propertydict:
            return propertydict["position"] / 64
        
# DEFINE MODEL
tracker = convolutional(input_shape=(64, 64, 1), number_of_outputs=2)

# DEFINE GENERATOR
generator = Generator().generate(training_set, get_position, shape=(96, 96), batch_size=32)



In [ ]:
# TRAIN TRACKER
tracker.fit(generator, epochs=100, steps_per_epoch=10)

In [ ]:
batch, labels = next(generator)
predictions = tracker.predict(batch) * 64
for image, position, prediction in zip(batch, labels, predictions):
    plt.imshow(image[:, :, 0], cmap='gray')
    plt.scatter(position[0], position[1], c='g', marker='x')
    plt.scatter(prediction[0], prediction[1], marker='o', facecolors=None, edgecolors='b')
    plt.show()